# Machine Learning Trader_Binance API

1. Setup

In [ ]:
apikey = 'BinanceAPIKEY'
secret = 'BinanceAPISECRET'

In [ ]:
!pip install python-binance pandas mplfinance

In [ ]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd

2. Authenticate

In [ ]:
client = Client(apikey, secret)

3. Get Tickers

In [ ]:
tickers = client.get_all_tickers()

In [ ]:
tickers

In [ ]:
tickers[1]['price']

In [ ]:
ticker_df = pd.DataFrame(tickers)

In [ ]:
ticker_df.head()

In [ ]:
ticker_df.tail()

In [ ]:
ticker_df.set_index('symbol', inplace=True)

In [ ]:
float(ticker_df.loc['ETHBTC']['price'])

4. Get Depth

In [ ]:
depth = client.get_order_book(symbol='BTCUSDT')

In [ ]:
depth

In [ ]:
depth_df = pd.DataFrame(depth['asks'])
depth_df.columns = ['Price', 'Volume']
depth_df.head()

In [ ]:
depth_df.dtypes

5. Get Historical Data

In [ ]:
#  [
#   [
#     1499040000000,      // Open time
#     "0.01634790",       // Open
#     "0.80000000",       // High
#     "0.01575800",       // Low
#     "0.01577100",       // Close
#     "148976.11427815",  // Volume
#     1499644799999,      // Close time
#     "2434.19055334",    // Quote asset volume
#     308,                // Number of trades
#     "1756.87402397",    // Taker buy base asset volume
#     "28.46694368",      // Taker buy quote asset volume
#     "17928899.62484339" // Ignore.
#   ]
# ]

In [ ]:
client.get_historical_klines??

In [ ]:
historical = client.get_historical_klines('ETHBTC', Client.KLINE_INTERVAL_1DAY, '1 Jan 2022')

In [ ]:
historical

In [ ]:
hist_df = pd.DataFrame(historical)

In [ ]:
hist_df.head()

In [ ]:
hist_df.columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                    'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']

In [ ]:
hist_df.tail()

In [ ]:
hist_df.shape

In [ ]:
hist_df.dtypes

6. Preprocess Historical Data

In [ ]:
hist_df['Open Time'] = pd.to_datetime(hist_df['Open Time']/1000, unit='s')
hist_df['Close Time'] = pd.to_datetime(hist_df['Close Time']/1000, unit='s')

In [ ]:
numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume']

In [ ]:
hist_df[numeric_columns] = hist_df[numeric_columns].apply(pd.to_numeric, axis=1)

In [ ]:
hist_df.tail()

In [ ]:
hist_df.dtypes

In [ ]:
hist_df.describe()

In [ ]:
hist_df.info()

7. Visualization

In [ ]:
import mplfinance as mpf

In [ ]:
hist_df.set_index('Close Time').tail(100)

In [ ]:
mpf.plot(hist_df.set_index('Close Time').tail(120), 
        type='candle', style='charles', 
        volume=True, 
        title='ETHBTC Last 120 Days', 
        mav=(10,20,30))